# Ejemplo de `word2vec` con `gensim`


En la siguiente celda, importamos las librerías necesarias y configuramos los mensajes de los logs.

In [1]:
import gensim, logging, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Entrenamiento de un modelo

Implemento una clase `Corpus` con un iterador sobre un directorio que contiene ficheros de texto. Utilizaré una instancia de `Corpus` para poder procesar de manera más eficiente una colección, sin necesidad de cargarlo previamente en memoria.

In [ ]:
class Corpus(object):
    '''Clase Corpus que permite leer de manera secuencial un directorio de documentos de texto'''
    
    def __init__(self, directorio):
        self.directory = directorio

    def __iter__(self):
        for fichero in os.listdir(self.directory):
            for linea in open(os.path.join(self.directory, fichero)):
                yield linea.split()


`CORPUSDIR` contiene una colección de noticias en español (normalizada previamente a minúsculas y sin signos de puntuación) con alrededor de 150 millones de palabras. Entrenamos un modelo en un solo paso, ignorando aquellos tokens que aparecen menos de 10 veces, para descartar erratas. 

In [ ]:
CORPUSDIR = 'PATH_TO_YOUR_CORPUS_DIRECTORY'
oraciones = Corpus(CORPUSDIR)
model = gensim.models.Word2Vec(oraciones, min_count=10, size=150, workers=2)

# el modelo puede entrenarse en dos pasos sucesivos pero por separado
#model = gensim.models.Word2Vec() # modelo vacío
#model.build_vocab(oraciones)  # primera pasada para crear la lista de vocabulario
#model.train(other_sentences)  # segunda pasada para calcula vectores

Una vez completado el entrenamiento (después de casi 30 minutos), guardamos el modelo en disco. 

In [ ]:
model.save('PATH_TO_YOUR_MODEL.w2v')

En el futuro, podremos utilizar este modelo cargándolo en memoria con la instrucción:

In [2]:
#model = gensim.models.Word2Vec.load('PATH_TO_YOUR_MODEL.w2v')
#model = gensim.models.Word2Vec.load('/data/w2v/eswiki-280.w2v')
model = gensim.models.Word2Vec.load('/data/w2v/efe-150.w2v')

2018-02-23 15:44:48,139 : INFO : loading Word2Vec object from /data/w2v/efe-150.w2v
2018-02-23 15:44:49,605 : INFO : loading wv recursively from /data/w2v/efe-150.w2v.wv.* with mmap=None
2018-02-23 15:44:49,607 : INFO : loading vectors from /data/w2v/efe-150.w2v.wv.vectors.npy with mmap=None
2018-02-23 15:44:49,754 : INFO : setting ignored attribute vectors_norm to None
2018-02-23 15:44:49,756 : INFO : loading vocabulary recursively from /data/w2v/efe-150.w2v.vocabulary.* with mmap=None
2018-02-23 15:44:49,756 : INFO : loading trainables recursively from /data/w2v/efe-150.w2v.trainables.* with mmap=None
2018-02-23 15:44:49,757 : INFO : loading syn1neg from /data/w2v/efe-150.w2v.trainables.syn1neg.npy with mmap=None
2018-02-23 15:44:49,903 : INFO : setting ignored attribute cum_table to None
2018-02-23 15:44:49,905 : INFO : loaded /data/w2v/efe-150.w2v


## Probando nuestro modelo

El objeto `model` contiene una enorme matriz de números: una tabla, donde cada fila es uno de los términos del vocabulario reconocido y cada columna es una de las características que permiten modelar el significado de dicho término.

En nuestro modelo, tal y como está entrenado, tenemos más de 26 millones de términos:

In [3]:
print(model.corpus_count)

26511278


Cada término del vocabulario está representado como un vector con 150 dimensiones: 105 características. Podemos acceder al vector de un término concreto:

In [4]:
print(model['azul'], '\n')

print(model['verde'], '\n')

print(model['microsoft'])

[ 0.23375803  0.7407618   0.62846696 -0.2483204  -1.3737258  -0.590827
  0.6523226   3.4684708   1.7477489  -1.3163776   0.82081383  0.20853145
  0.7314245   0.13800569  1.1055775  -0.57149947 -1.1715975  -1.193085
 -0.58984065  1.0502697   0.74993986  0.34181598 -0.0588204   0.4395582
  0.76400846 -1.9379612   1.1530447   0.44922572  0.82149845  0.26971048
 -0.38326746 -1.758451   -0.5402964  -0.98902136 -1.1511512  -1.4542484
 -2.0532486  -0.84603715  1.1789035   0.8954198  -0.07105764 -0.6009114
 -0.32276174 -0.5073711  -0.72784954 -1.0411621   0.7728746   0.9850062
  1.5772941  -1.6976876   1.010405   -0.8032998   1.3950039   1.461288
  1.6583639  -0.01957864  0.72737986  1.2166173  -0.7563755  -0.49769273
 -1.0798682   0.7831595  -0.7357984   2.294056   -0.3698049  -2.173252
  0.9841621   0.27959237 -0.12817219  0.48054722  1.4299809  -1.2538192
  1.4374963   0.7781263  -0.33234715 -0.8497048  -0.21231754  2.2888587
 -1.5101597   0.5487344  -1.305364   -0.4762724  -2.3166475  -0.5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


KeyError: "word 'microsoft' not in vocabulary"

Estos vectores no nos dicen mucho, salvo que contienen números muy pequeños :-/

El mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [8]:
print('hombre - mujer', model.wv.similarity('hombre', 'mujer'))

print('perro - gato', model.wv.similarity('perro', 'gato'))

print('gato - periódico', model.wv.similarity('gato', 'periódico'))

hombre - mujer 0.4590373348196588
perro - gato 0.8069666086573097
gato - periódico 0.3945044491661706


Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [10]:
lista1 = 'madrid barcelona gonzález washington'.split()
print('en la lista', ' '.join(lista1), 'sobra:', model.wv.doesnt_match(lista1))

#lista2 = 'psoe pp ciu epi'.split()
#print('en la lista', ' '.join(lista2), 'sobra:', model.wv.doesnt_match(lista2))

lista3 = 'publicaron declararon soy negaron'.split()
print('en la lista', ' '.join(lista3), 'sobra:', model.wv.doesnt_match(lista3))

lista3 = 'homero saturno cervantes shakespeare cela'.split()
print('en la lista', ' '.join(lista3), 'sobra:', model.wv.doesnt_match(lista3))

2018-02-23 15:46:24,991 : WARNING : vectors for words {'gonzález', 'washington'} are not present in the model, ignoring these words
2018-02-23 15:46:24,997 : WARNING : vectors for words {'pp', 'psoe', 'ciu', 'epi'} are not present in the model, ignoring these words


en la lista madrid barcelona gonzález washington sobra: barcelona


ValueError: cannot select a word from an empty list

Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [11]:
terminos = 'psoe chicago sevilla aznar podemos estuvieron'.split()
terminos = 'microsoft ibm iberia repsol'.split()

for t in terminos:
    print(t, '==>', model.wv.most_similar(t), '\n')

KeyError: "word 'microsoft' not in vocabulary"

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [ ]:
print('==> alcalde + mujer - hombre')
most_similar = model.wv.most_similar(positive=['alcalde', 'mujer'], negative=['hombre'], topn=3)
for item in most_similar:
    print(item)

print('==> madrid + filipinas - españa')
most_similar = model.wmost_similar(positive=['madrid', 'filipinas'], negative=['españa'], topn=3)
for item in most_similar:
    print(item)

    
    